In [1]:
%matplotlib inline
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import gc
import time
from tqdm import tqdm
import cartopy.crs as ccrs
import matplotlib.ticker as mticker

def open_nc(path,scale=1.0):  
    ds=(xr.open_dataset(path)*scale)
    ds=ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180))
    ds=ds.reindex(lon=sorted(ds.lon))  
    return ds

In [2]:
vari_ls =[
    "bc_a1_SRF","bc_a4_SRF",
    "dst_a1_SRF","dst_a2_SRF",
    "ncl_a1_SRF","ncl_a2_SRF",
    "pom_a1_SRF","pom_a4_SRF",
    "so4_a1_SRF","so4_a2_SRF",
    "soa_a1_SRF","soa_a2_SRF"
]

path = "/data/keeling/a/zzheng25/d/mam4_paper_data/mam4_cesm_cal/"
save_path = "/data/keeling/a/zzheng25/d/mam4_paper_data/chi_only/comp_analysis/"

## For year

In [3]:
for year in ["2011"]:
    month_ls = []
    for i in tqdm(range(1,13)):
        month=str(i).zfill(2)
        ds=xr.open_dataset(path+str(year)+"_"+month+".nc")
        month_ls.append(ds[vari_ls].mean(dim="time")\
                       .assign_coords(month_idx=month)\
                       .expand_dims('month_idx'))
        del ds
        gc.collect()

    ds_m = xr.merge(month_ls).mean(dim="month_idx")
    del month_ls
    gc.collect()

    # create an the mass all, then calculate the percentage
    ## create an emtpy mass all
    ds_m["Mass_all"] = xr.DataArray(np.zeros(ds_m["bc_a1_SRF"].shape), 
                                    coords=[ds_m.lat, ds_m.lon], 
                                    dims=['lat', 'lon'])
    ## add variables
    for vari in tqdm(vari_ls):
        ds_m["Mass_all"] += ds_m[vari]

    ## calculate the percentage
    for vari in tqdm(vari_ls):
        ds_m[vari[:-4]+"_per"] = ds_m[vari]/ds_m["Mass_all"]*100.0


    ds_m.to_netcdf(save_path+str(year)+"_"+"year_comp.nc")
    del ds_m
    gc.collect()

100%|██████████| 12/12 [00:00<00:00, 332.47it/s]
